<a href="https://colab.research.google.com/github/mridul-eecs/signal-processing-apachesparkml-apachesystemml/blob/master/LogisticRegression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:

# spark dependencies:
# citation: http://medium.com/@rmache/big-data-with-spark-in-google-colab-7c046e24b3
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://apachemirror.wuchna.com/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
!tar xf spark-2.4.4-bin-hadoop2.7.tgz

!pip install -q findspark
!pip install pyspark

     |████████████████████████████████| 215.7MB 55kB/s 
     |████████████████████████████████| 204kB 44.9MB/s 
  Created wheel for pyspark: filename=pyspark-2.4.4-py2.py3-none-any.whl size=216130387 sha256=1616c4161ac23a6326fb6b3b1257bb69bbd425da4f479ed271941211e4b64b02
  Stored in directory: /root/.cache/pip/wheels/ab/09/4d/0d184230058e654eb1b04467dbc1292f00eaa186544604b471
Successfully built pyspark


In [0]:
!git clone https://github.com/mridul-eecs/signal-processing-apachesparkml-apachesystemml.git

Cloning into 'signal-processing-apachesparkml-apachesystemml'...
remote: Enumerating objects: 19, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 19 (delta 1), reused 14 (delta 1), pack-reused 0
Unpacking objects: 100% (19/19), done.


In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructField, StructType, IntegerType
from pyspark.sql.functions import lit
import os
from tqdm import tqdm_notebook as tqdm

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"


import findspark
findspark.init()

APP_NAME= "Human Motion Premitives"
SPARK_URL= "local[*]"
RANDOM_SEED = 141109
TRAINING_DATA_RATIO = 0.7
RF_NUM_TREES = 8
RF_MAX_DEPTH = 4
RF_NUM_BINS = 32

In [0]:
spark= SparkSession.builder.appName(APP_NAME).master(SPARK_URL).getOrCreate()

In [0]:
df= spark.read.parquet('/content/signal-processing-apachesparkml-apachesystemml/df.parquet')
df.show()

+---+---+---+-------------+
|  x|  y|  z|        class|
+---+---+---+-------------+
| 17| 46| 41|    Getup_bed|
| 31| 41| 51|  Drink_glass|
| 31| 41| 51|  Drink_glass|
| 31| 41| 51|  Drink_glass|
| 31| 41| 51|  Drink_glass|
| 31| 41| 51|  Drink_glass|
| 31| 41| 51|  Drink_glass|
| 31| 41| 51|  Drink_glass|
| 31| 41| 51|  Drink_glass|
| 31| 41| 51|  Drink_glass|
| 31| 41| 51|  Drink_glass|
| 28| 41| 36|  Brush_teeth|
| 34| 53| 33|         Walk|
| 14| 46| 31|Standup_chair|
| 18| 45| 36|Sitdown_chair|
| 14| 26| 39|  Liedown_bed|
| 28| 41| 48|         Walk|
| 26| 52| 32|  Brush_teeth|
| 26| 52| 32|  Brush_teeth|
| 26| 52| 32|  Brush_teeth|
+---+---+---+-------------+
only showing top 20 rows



In [0]:
df_train, df_test= df.randomSplit([0.8, 0.2])

In [0]:
df_train.createOrReplaceTempView('df_train')

from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline

indexer= StringIndexer(inputCol= 'class', outputCol= 'label')
vectorizer= VectorAssembler(inputCols= ['x', 'y', 'z'], outputCol= 'features')
classifier= LogisticRegression(maxIter= 10, regParam= 0.3, elasticNetParam= 0.8)

pipe= Pipeline(stages= [indexer, vectorizer, classifier])
model= pipe.fit(df_train)
pred= model.transform(df_train)

In [0]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator as Evaluator
eval= Evaluator().setMetricName('accuracy').setLabelCol('label').setPredictionCol('prediction')
eval.evaluate(pred)

0.20636972617228816

In [0]:
pred= model.transform(df_test)
eval= Evaluator().setMetricName('accuracy').setLabelCol('label').setPredictionCol('prediction')
eval.evaluate(pred)

0.20753552960707114